# Azure AI Search Blob knowledge source sample

This Python notebook demonstrates the [blob knowledge source](https://learn.microsoft.com/azure/search/search-knowledge-source-how-to-blob) feature of Azure AI Search that is currently in public preview.

Blob knowledge source takes a dependency on [integrated vectorization](https://learn.microsoft.com/azure/search/vector-search-integrated-vectorization). It automatically:
* Generates a data source, skillset, indexer, and index to represent the underlying blob content in Azure AI Search
* Connects an embedding model and chat completion model you bring with a skillset using any of the following skills:
   * [Azure OpenAI Embedding skill](https://learn.microsoft.com/azure/search/cognitive-search-skill-azure-openai-embedding)
   * [GenAI Prompt skill](https://learn.microsoft.com/azure/search/cognitive-search-skill-genai-prompt)
   * [Azure AI Vision multimodal embeddings skill](https://learn.microsoft.com/azure/search/cognitive-search-skill-vision-vectorize)
   * [AML Skill](https://learn.microsoft.com/azure/search/cognitive-search-aml-skill)

These example uses PDFs from the `data/documents` folder to create a blob knowledge source. A knowledge agent is then used to chat with these PDFs.

## Prerequisites
+ An Azure subscription, with [access to Azure OpenAI](https://aka.ms/oai/access).
 
+ Azure AI Search, Basic or higher for this workload. [Semantic ranker](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable) is required for [agentic retrieval](https://learn.microsoft.com/azure/search/search-agentic-retrieval-concept) and it's not available on the free tier.

+ A deployment of the `text-embedding-3-large` model on Azure OpenAI.

+ A deployment of the `gpt-5` model on Azure OpenAI.

+ Azure Blob Storage. This notebook connects to your storage account and loads a container with the sample PDFs.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [ ]:
! pip install -r requirements.txt --quiet

### Set and load environment variables

1. Copy .env-sample from the demo-python folder to the knowledge subfolder.
1. Rename the file to `.env`.
1. Set the knowledge variables to your Azure resources, API keys, connections strings, and provide names for the new objects created by the code.
1. Execute the cell to load the environment variables.

This example includes image verbalization that uses an LLM to describe embedded images in your source content. You can enable image verbalization by setting `USE_VERBALIZATION` to true in the `.env` file.

In [1]:
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()
knowledge_source_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_SOURCE", "blob-knowledge-source")
knowledge_agent_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_AGENT", "blob-knowledge-agent")
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
# search blob datasource connection string is optional - defaults to blob connection string
# This field is only necessary if you are using MI to connect to the data source
# https://learn.microsoft.com/azure/search/search-howto-indexing-azure-blob-storage#supported-credentials-and-connection-strings
search_blob_connection_string = os.getenv("SEARCH_BLOB_DATASOURCE_CONNECTION_STRING", blob_connection_string)
blob_container_name = os.getenv("BLOB_CONTAINER_NAME", "documents")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME", "text-embedding-3-large")
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-5")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-5")
use_verbalization = os.getenv("USE_VERBALIZATION", "false") == "true"


## Connect to Blob Storage and load documents

Retrieve documents from Blob Storage. You can use the sample documents in the data/documents folder.  

In [ ]:
from azure.storage.blob.aio import BlobServiceClient  
import glob

sample_docs_directory = os.path.join("..", "..", "..", "data", "benefitdocs")

async def upload_sample_documents(
        blob_connection_string: str,
        blob_container_name: str,
        documents_directory: str,
        # Set to false if you want to use credentials included in the blob connection string
        # Otherwise your identity will be used as credentials
        use_user_identity: bool = True
    ):
        # Connect to Blob Storage
        async with DefaultAzureCredential() as user_credential, BlobServiceClient.from_connection_string(logging_enable=True, conn_str=blob_connection_string, credential=user_credential if use_user_identity else None) as blob_service_client:
            async with blob_service_client.get_container_client(blob_container_name) as container_client:
                if not await container_client.exists():
                    await container_client.create_container()

                files = glob.glob(os.path.join(documents_directory, '*'))
                for file in files:
                    with open(file, "rb") as data:
                        name = os.path.basename(file)
                        async with container_client.get_blob_client(name) as blob_client:
                            if not await blob_client.exists():
                                await blob_client.upload_blob(data)

docs_directory = sample_docs_directory

await upload_sample_documents(
    blob_connection_string = blob_connection_string,
    blob_container_name = blob_container_name,
    documents_directory = docs_directory)

print(f"Setup sample data in {blob_container_name}")

Setup sample data in documents


## Create a blob knowledge source on Azure AI Search

Creating a [blob knowledge source](https://learn.microsoft.com/azure/search/search-knowledge-source-how-to-blob) sets up all necessary resources to index the uploaded documents.

In [3]:
from azure.search.documents.indexes.models import AzureBlobKnowledgeSource, AzureBlobKnowledgeSourceParameters, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, KnowledgeAgentAzureOpenAIModel
from azure.search.documents.indexes.aio import SearchIndexClient

chat_model = KnowledgeAgentAzureOpenAIModel(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=azure_openai_endpoint,
        deployment_name=azure_openai_chatgpt_deployment,
        api_key=azure_openai_key,
        model_name=azure_openai_chatgpt_model_name
    )
)

knowledge_source = AzureBlobKnowledgeSource(
    name=knowledge_source_name,
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=search_blob_connection_string,
        container_name=blob_container_name,
        embedding_model=AzureOpenAIVectorizer(
            vectorizer_name="blob-vectorizer",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,
                deployment_name=azure_openai_embedding_deployment,
                api_key=azure_openai_key,
                model_name=azure_openai_embedding_model_name
            )
        ),
        chat_completion_model=chat_model if use_verbalization else None,
        disable_image_verbalization=not use_verbalization
    )
)

async with SearchIndexClient(endpoint=endpoint, credential=credential) as client:
    await client.create_or_update_knowledge_source(knowledge_source)
    print(f"Created knowledge source: {knowledge_source.name}")

Created knowledge source: blob-knowledge-source


## Create a knowledge agent on Azure AI Search

This step creates a knowledge agent, which acts as a wrapper for your knowledge source and LLM deployment.

`EXTRACTIVE_DATA` is the default modality and returns content from your knowledge sources without generative alteration. Use the `ANSWER_SYNTHESIS` modality for [LLM-generated answers that cite the retrieved content](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-synthesize).

In [4]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeSourceReference, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality

output_config = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,
    include_activity=True
)

agent = KnowledgeAgent(
    name=knowledge_agent_name,
    models=[chat_model],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source.name,
            include_reference_source_data=True,
            always_query_source=True
        )
    ],
    output_configuration=output_config
)

async with SearchIndexClient(endpoint=endpoint, credential=credential) as index_client:
    await index_client.create_or_update_agent(agent)
    print(f"Created knowledge agent: {agent.name}")

Created knowledge agent: blob-knowledge-agent


## Use agentic retrieval to fetch results

This step runs the agentic retrieval pipeline to produce a grounded, citation-backed answer. Given the conversation history and retrieval parameters, your knowledge agent:

* Analyzes the entire conversation to infer the user's information need.
* Decomposes the compound query into focused subqueries.
* Executes the subqueries concurrently against your knowledge source.
* Uses semantic ranker to rerank and filter the results.
* Synthesizes the top results into a natural-language answer.

In [5]:
from azure.search.documents.agent.aio import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, SearchIndexKnowledgeSourceParams

messages = [
    KnowledgeAgentMessage(
        role="user",
        content=[KnowledgeAgentMessageTextContent(
            text="Differences between Northwind Health Plus and Standard"
        )]
    )
]

agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=knowledge_agent_name, credential=credential)
result = await agent_client.retrieve(KnowledgeAgentRetrievalRequest(messages=messages))

## Review the retrieval response, activity, and results
Because your knowledge agent is configured for answer synthesis, the retrieval response contains the following values:

* `response_content`: An LLM-generated answer to the query that cites the retrieved documents.
* `activity_content`: Detailed planning and execution information, including subqueries, reranking decisions, and intermediate steps.
* `references_content`: Source documents and chunks that contributed to the answer.

*Tip:* Retrieval parameters, such as reranker thresholds and knowledge source parameters, influence how aggressively your agent reranks and which sources it queries. Inspect the activity and references to validate grounding and build traceable citations.


In [6]:
print(result.response[0].content[0].text)

Northwind Health Plus and Northwind Standard differ in several key areas:

- **Coverage Scope**: Northwind Health Plus is a comprehensive plan that covers medical, vision, and dental services, as well as prescription drugs, mental health and substance abuse services, preventive care, and emergency services (both in-network and out-of-network). Northwind Standard is a more basic plan that covers medical, vision, and dental services, preventive care, and prescription drugs, but does not cover emergency services, mental health and substance abuse services, or out-of-network services [ref_id:0][ref_id:2][ref_id:1].

- **Prescription Drugs**: Northwind Health Plus covers a wider range of prescription drugs, including generic, brand-name, and specialty drugs. Northwind Standard only covers generic and brand-name drugs [ref_id:0].

- **Vision and Dental**: Both plans cover vision and dental services, but Northwind Health Plus includes coverage for vision exams, glasses, contact lenses, and de

In [7]:
import json

# Activity -> JSON string of activity as list of dicts

activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 913,
    "input_tokens": 2021,
    "output_tokens": 48
  },
  {
    "id": 1,
    "type": "azureBlob",
    "elapsed_ms": 555,
    "knowledge_source_name": "blob-knowledge-source",
    "query_time": "2025-09-09T05:40:40.117Z",
    "count": 50,
    "azure_blob_arguments": {
      "search": "Differences between Northwind Health Plus and Northwind Health Standard plans"
    }
  },
  {
    "id": 2,
    "type": "semanticReranker",
    "input_tokens": 0
  },
  {
    "id": 3,
    "type": "modelAnswerSynthesis",
    "elapsed_ms": 3968,
    "input_tokens": 7010,
    "output_tokens": 402
  }
] 



In [ ]:
# References -> JSON string of references as list of dicts
references_content = json.dumps([r.as_dict() for r in result.references], indent=2)
print("references_content:\n", references_content, "\n")